In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.cluster import KMeans
from scipy.optimize import minimize, differential_evolution
from scipy.spatial.distance import pdist, squareform
import pandas as pd
from manifold_learning import *

In [3]:

fitter = ManifoldFitter()

print("=== Fitting Real Data onto Manifolds ===\n")

# Load sample datasets
datasets = fitter.load_sample_datasets()

for dataset_name, dataset_info in datasets.items():
    print(f"\n--- Analyzing {dataset_name}: {dataset_info['description']} ---")

    data = dataset_info['data']

    # Preprocessing
    if data.shape[1] > 3:
        # Reduce dimension to 3D
        pca = PCA(n_components=3)
        data_3d = pca.fit_transform(data)
        print(f"Dimension reduction: {data.shape[1]} -> 3")
        print(f"Variance preserved: {sum(pca.explained_variance_ratio_):.3f}")
    else:
        data_3d = data

    # Standardize
    data_scaled, scaler = fitter.preprocess_data(data_3d, method='standard')

    # Estimate the best manifold
    best_manifold, scores = fitter.estimate_manifold_type(data_scaled)
    print(f"\nManifold scores:")
    for manifold, score in scores.items():
        print(f"  {manifold}: {score:.3f}")
    print(f"Suggestion: {best_manifold}")

    # Fit to all manifolds
    results, sorted_results = fitter.fit_data_to_manifolds(data_scaled)

    print(f"\nFit results (sorted by error):")
    for manifold, result in sorted_results:
        if 'error' in result and result['error'] != float('inf'):
            print(f"  {manifold}: Error = {result['error']:.6f}")
            if 'aic' in result:
                print(f"    AIC = {result['aic']:.2f}, BIC = {result['bic']:.2f}")

    # Display parameters of the best fit
    if sorted_results:
        best_manifold_result = sorted_results[0]
        manifold_name = best_manifold_result[0]
        manifold_data = best_manifold_result[1]

        if 'params' in manifold_data:
            print(f"\nParameters for {manifold_name}:")
            for param_name, param_value in manifold_data['params'].items():
                if isinstance(param_value, np.ndarray):
                    print(f"  {param_name}: {param_value}")
                else:
                    print(f"  {param_name}: {param_value:.4f}")

    # Plot results (optional)
    # fitter.visualize_fits(data_scaled, results, dataset_name)

    print("-" * 60)

print("\n=== Example with Custom Data ===")

# Generate synthetic data for testing
np.random.seed(42)

# Spherical data
theta = np.random.uniform(0, np.pi, 300)
phi = np.random.uniform(0, 2*np.pi, 300)
sphere_data = np.column_stack([
    2 * np.sin(theta) * np.cos(phi) + np.random.normal(0, 0.1, 300),
    2 * np.sin(theta) * np.sin(phi) + np.random.normal(0, 0.1, 300),
    2 * np.cos(theta) + np.random.normal(0, 0.1, 300)
])

print("Testing with spherical data:")
results, sorted_results = fitter.fit_data_to_manifolds(sphere_data)

for manifold, result in sorted_results:
    if 'error' in result and result['error'] != float('inf'):
        print(f"  {manifold}: Error = {result['error']:.6f}")

# Check if the fit is correct
best_result = sorted_results[0]
if best_result[0] == 'sphere':
    sphere_params = best_result[1]['params']
    estimated_center = sphere_params['center']
    estimated_radius = sphere_params['radius']

    print(f"\nEstimated center: {estimated_center}")
    print(f"Estimated radius: {estimated_radius:.3f}")
    print("(Expected: center ≈ [0,0,0], radius ≈ 2)")

=== Fitting Real Data onto Manifolds ===


--- Analyzing iris: Iris flower - 4 features, 3 classes ---
Dimension reduction: 4 -> 3
Variance preserved: 0.995

Manifold scores:
  sphere: 0.733
  torus: 1.000
  disk: 0.667
  plane: 0.533
  swiss_roll: 0.974
Suggestion: torus

Fit results (sorted by error):
  torus: Error = 0.232356
    AIC = -208.92, BIC = -193.87
  sphere: Error = 0.313099
    AIC = -166.19, BIC = -154.14
  disk: Error = 1.000000
    AIC = 16.00, BIC = 40.09

Parameters for torus:
  center: [ 0.98054178 -0.16026558  0.15088943]
  R: 1.4938
  r: 1.2433
------------------------------------------------------------

--- Analyzing digits: Handwritten digits - 64 features, 10 classes ---
Dimension reduction: 64 -> 3
Variance preserved: 0.403

Manifold scores:
  sphere: 0.766
  torus: 1.000
  disk: 0.667
  plane: 0.533
  swiss_roll: 0.882
Suggestion: torus

Fit results (sorted by error):
  sphere: Error = 0.226471
    AIC = -2660.79, BIC = -2638.82
  torus: Error = 0.246498
   